# 문장 유사도 예측 (두 문장 비교)

In [ ]:
import tensorflow as tf #1
from tensorflow.keras.layers import Input, Embedding, Dense #2 
from tensorflow.keras.models import Model #3
#1) tensorflow : 딥러닝 프레임워크
#2) Input, Embedding, Dense : 모델 구성에 필요한 층
#3) Model : 전체 모델을 정의하기 위해 사용

In [ ]:
# 1. 입력 정의
input1 = Input(shape=(10,), name='input1')
input2 = Input(shape=(10,), name='input2')
# shape=(10,): 각 문장은 최대 10개의 단어(정수로 바뀐)로 구성돼 있다고 가정
# Input(): 모델의 입력을 정의

embedding = Embedding(input_dim=1000, output_dim=32, name='shared_embedding')
# input_dim=1000: 전체 단어 사전 크기 (즉, 정수 인덱스는 0~999까지 가능)
# output_dim=32: 각 단어를 32차원 벡터로 바꿈
# 공통 임베딩 레이어를 쓰는 이유: 두 문장을 같은 기준으로 표현하기 위해서야 (공정한 비교)

In [ ]:
# 2. 임베딩 적용
x1 = embedding(input1)
x2 = embedding(input2)
# input1, input2에 embedding 레이어를 적용해서 벡터 형태로 변환
# 두 문장은 이제 (batch_size, 10, 32) 크기의 벡터 시퀀스가 됨

In [ ]:
!python -m pip install keras-nlp

In [ ]:
# 3. Transformer Encoder 적용
import keras_nlp

encoder_layer = keras_nlp.layers.TransformerEncoder(
    intermediate_dim=64,
    num_heads=2,
)
x1 = encoder_layer(x1)
x2 = encoder_layer(x2)
# TransformerEncoder: 문장 내 단어들 사이 관계를 파악함 (자기 주의, Self-Attention)
# num_heads=2: attention을 2개의 관점에서 보겠다는 의미
# intermediate_dim=64: 내부 feed-forward 네트워크 크기
# 두 문장에 같은 인코더를 쓰는 이유 : 같은 방식으로 의미를 추출해서 비교할 수 있게 하기위함

In [ ]:
# 4. 문장 요약 (Pooling)
from tensorflow.keras.layers import Lambda

x1 = Lambda(lambda x: tf.reduce_mean(x, axis=1))(x1)
x2 = Lambda(lambda x: tf.reduce_mean(x, axis=1))(x2)
# 각 문장의 단어 벡터들을 평균 내서 하나의 문장 벡터로 축약
# shape: (batch_size, 32) → 한 문장을 대표하는 벡터

In [ ]:
# 5. 두 문장 비교 (차이 계산)
from tensorflow.keras.layers import Lambda

x = Lambda(lambda tensors: tf.abs(tensors[0] - tensors[1]))([x1, x2])
# 절댓값 차이를 계산하면 두 문장의 벡터가 얼마나 다른지를 수치로 볼 수 있음
# 차이가 작으면 비슷한 문장, 크면 다른 문장

In [ ]:
# 6. 출력층 구성 (시그모이드)
from tensorflow.keras.layers import Dense

output = Dense(1, activation='sigmoid')(x)
# 시그모이드는 0~1 사이 값을 줌 → 유사도 확률로 해석 가능
# 1에 가까울수록 비슷한 문장
# Dense(1) : 출력 뉴런 1개 (→ 유사도 점수 하나만 출력함)
# activation='sigmoid' : 0~1 사이 확률로 출력되게 함
# (x) : 이전 단계에서 나온 두 문장의 차이값


In [ ]:
# 7. 모델 정의 & 컴파일
from tensorflow.keras.models import Model

model = Model(inputs=[input1, input2], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
# Model: 입력 2개, 출력 1개인 구조 정의
# optimizer='adam' : 가장 많이 쓰는 옵티마이저
# binary_crossentropy: 두 문장이 같은지(1), 다른지(0)를 판단하는 이진 분류 문제이기 때문
# metrics=['accuracy'] : 정확도 확인용
# adam: 최적화 알고리즘 (학습을 효율적으로 해줌)

In [ ]:
# 8. 예측 테스트
import numpy as np

def predict_similarity(sentence1, sentence2):
    # 두 문장을 정수 시퀀스로 변환
    seq1 = tokenizer.texts_to_sequences([sentence1])
    seq2 = tokenizer.texts_to_sequences([sentence2])
    
    # 패딩 (입력 길이에 맞게 0으로 채워줌)
    seq1 = tf.keras.preprocessing.sequence.pad_sequences(seq1, maxlen=10)
    seq2 = tf.keras.preprocessing.sequence.pad_sequences(seq2, maxlen=10)

    # 예측
    pred = model.predict([seq1, seq2])
    
    print(f'유사도 예측 결과: {pred[0][0]:.4f}')

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf

# 1. 훈련 문장 준비 (예시)
sentences = [
    "나는 영화를 좋아해",
    "너는 축구를 좋아하니?",
    "오늘 날씨가 좋다",
    "나는 영화를 정말 좋아해",
    "축구는 재미있어"
]

# 2. 토크나이저 생성 및 훈련
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

# 3. 최대 길이 정의
max_len = max(len(s.split()) for s in sentences)

In [ ]:
predict_similarity("나는 영화를 좋아해", "나는 영화를 정말 좋아해")